#  aitextgen — Train a GPT-2 (or GPT Neo) Text-Generating Model w/ GPU

by [Max Woolf](https://minimaxir.com)

*Last updated: May 16th, 2021 (aitextgen v0.5.2)*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Colaboratory** using `aitextgen`!

For more about `aitextgen`, you can visit [this GitHub repository](https://github.com/minimaxir/aitextgen) or [read the documentation](https://docs.aitextgen.io/).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Run the cells below:


In [ ]:
!pip install -q aitextgen

     |████████████████████████████████| 572 kB 23.7 MB/s 
     |████████████████████████████████| 4.2 MB 54.0 MB/s 
     |████████████████████████████████| 87 kB 1.9 MB/s 
     |████████████████████████████████| 585 kB 56.0 MB/s 
     |████████████████████████████████| 419 kB 53.8 MB/s 
     |████████████████████████████████| 140 kB 59.0 MB/s 
     |████████████████████████████████| 596 kB 52.9 MB/s 
     |████████████████████████████████| 1.1 MB 52.6 MB/s 
     |████████████████████████████████| 6.6 MB 62.1 MB/s 
     |████████████████████████████████| 86 kB 4.1 MB/s 
     |████████████████████████████████| 271 kB 34.0 MB/s 
     |████████████████████████████████| 144 kB 63.8 MB/s 
     |████████████████████████████████| 94 kB 1.2 MB/s 


In [ ]:
import aitextgen
import datetime
import gc
import logging
import os
import requests
import torch

In [ ]:
session_url = 'http://172.28.0.2:9000/api/sessions'
notebook_name = requests.get(session_url).json()[0]['name']

run_datetime = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
run_id = notebook_name + '_run_' + run_datetime

In [ ]:
log_format = '%(asctime)s — %(levelname)s — %(name)s — %(message)s'
date_format = '%d/%m/%Y %H:%M:%S'
log_level = logging.DEBUG

logging.basicConfig(format=log_format, datefmt=date_format, level=log_level)

## GPU

Colaboratory uses a Nvidia P4, an Nvidia T4, an Nvidia P100, or an Nvidia V100. For finetuning GPT-2 124M, any of these GPUs will be fine, but for text generation, a T4 or a P100 is ideal since they have more VRAM. **If you receive a T4 or a V100 GPU, you can enable `fp16=True` during training for faster/more memory efficient training.**

You can verify which GPU is active by running the cell below. If you want to try for a different GPU, go to **Runtime -> Factory Reset Runtime**.

In [ ]:
!nvidia-smi

Wed Jun  8 13:04:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
aitextgen.colab.mount_gdrive()

Mounted at /content/drive


In [ ]:
gdrive_root_dir = '/content/drive/My Drive'


## Load a Trained Model

If you already had a trained model from this notebook, running the next cell will copy the `pytorch_model.bin` and the `config.json`file from the specified folder in Google Drive into the Colaboratory VM. (If no `from_folder` is specified, it assumes the two files are located at the root level of your Google Drive)

In [ ]:
load_model = None
#load_model = 'aitextgen-CCS-124M-7200-3600_run_2022-05-06-18-01-15'

In [ ]:
if load_model is not None:
    model_load_dir = gdrive_root_dir + '/aitextgen/models/' + load_model
    ai = aitextgen.aitextgen(model_folder=model_load_dir, to_gpu=True)

The next cell will allow you to load the retrained model + metadata necessary to generate text.

## Loading GPT-2 or GPT Neo

If you're retraining a model on new text, you need to download and load the GPT-2 model into the GPU. 

There are several sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M` (default): the "medium" model, 1.5GB on disk.
* `774M` (default): the "large" model, 3GB on disk.

You can also finetune a GPT Neo model instead, which is more suitable for longer texts and the base model has more recent data:

* `125M`: Analogous to the GPT-2 124M model.
* `350M`: Analogous to the GPT-2 355M model

The next cell downloads the model and saves it in the Colaboratory VM. If the model has already been downloaded, running this cell will reload it.

In [ ]:
#model='124M'
#model='355M'
#model='774M'

model='EleutherAI/gpt-neo-125M'
#model='EleutherAI/gpt-neo-350M'

In [ ]:
if load_model is None:
    if model == '124M' or model == '355M' or model == '774M':
        ai = aitextgen.aitextgen(tf_gpt2=model, to_gpu=True)
    else:
        ai = aitextgen.aitextgen(model=model, to_gpu=True)

08/06/2022 13:04:45 — INFO — aitextgen — Downloading EleutherAI/gpt-neo-125M model to /aitextgen.
08/06/2022 13:04:45 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
08/06/2022 13:04:46 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/config.json not found in cache or force_download set to True, downloading to /content/aitextgen/tmp2m1aghxs
08/06/2022 13:04:46 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
08/06/2022 13:04:47 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "GET /EleutherAI/gpt-neo-125M/resolve/main/config.json HTTP/1.1" 200 1007


Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/config.json in cache at aitextgen/29380fef22a43cbfb3d3a6c8e2f4fd951459584d87c34e4621b30580a54aca84.f0f7ebddfc6e15a23ac33e7fa95cd8cca05edf87cc74f9e3be7905f538a59762
creating metadata file for aitextgen/29380fef22a43cbfb3d3a6c8e2f4fd951459584d87c34e4621b30580a54aca84.f0f7ebddfc6e15a23ac33e7fa95cd8cca05edf87cc74f9e3be7905f538a59762
08/06/2022 13:04:47 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
08/06/2022 13:04:48 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/pytorch_model.bin not found in cache or force_download set to True, downloading to /content/aitextgen/tmpxtglqgjz
08/06/2022 13:04:48 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): cdn-lfs.huggingface.co:443
08/06/2022 13:04:49 — DEBUG — urllib3.con

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/pytorch_model.bin in cache at aitextgen/b0ace3b93ace62067a246888f1e54e2d3ec20807d4d3e27ac602eef3b7091c0b.6525df88f1d5a2d33d95ce2458ef6af9658fe7d1393d6707e0e318779ccc68ff
creating metadata file for aitextgen/b0ace3b93ace62067a246888f1e54e2d3ec20807d4d3e27ac602eef3b7091c0b.6525df88f1d5a2d33d95ce2458ef6af9658fe7d1393d6707e0e318779ccc68ff
08/06/2022 13:05:22 — INFO — aitextgen — Using the tokenizer for EleutherAI/gpt-neo-125M.
08/06/2022 13:05:22 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
08/06/2022 13:05:23 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /content/aitextgen/tmpov7h78lj
08/06/2022 13:05:23 — DEBUG — urllib3.connectionpool —

...

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/merges.txt in cache at aitextgen/12305762709d884a770efe7b0c68a7f4bc918da44e956058d43da0d12f7bea20.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for aitextgen/12305762709d884a770efe7b0c68a7f4bc918da44e956058d43da0d12f7bea20.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
08/06/2022 13:05:32 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
08/06/2022 13:05:33 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/tokenizer.json HTTP/1.1" 404 0
08/06/2022 13:05:33 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
08/06/2022 13:05:34 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/added_tokens.json HTTP/1.1" 404 0
08/06/2022 13:05:34 — DEBUG — urllib3.connectionpool — Starting new

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/special_tokens_map.json in cache at aitextgen/6c3239a63aaf46ec7625b38abfe41fc2ce0b25f90800aefe6526256340d4ab6d.2b8bf81243d08385c806171bc7ced6d2a0dcc7f896ca637f4e777418f7f0cc3c
creating metadata file for aitextgen/6c3239a63aaf46ec7625b38abfe41fc2ce0b25f90800aefe6526256340d4ab6d.2b8bf81243d08385c806171bc7ced6d2a0dcc7f896ca637f4e777418f7f0cc3c
08/06/2022 13:05:36 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
08/06/2022 13:05:37 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/vocab.json from cache at aitextgen/08c00c4159e921d4c941ac75732643373aba509d9b352a82bbbb043a94058d98.a552555fdda56a1c7c9a285bccfd44ac8e4b9e26c8c9b307831b3ea3ac782b45
loading file https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/merge

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

storing https://huggingface.co/EleutherAI/gpt-neo-125M/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/29380fef22a43cbfb3d3a6c8e2f4fd951459584d87c34e4621b30580a54aca84.f0f7ebddfc6e15a23ac33e7fa95cd8cca05edf87cc74f9e3be7905f538a59762
creating metadata file for /root/.cache/huggingface/transformers/29380fef22a43cbfb3d3a6c8e2f4fd951459584d87c34e4621b30580a54aca84.f0f7ebddfc6e15a23ac33e7fa95cd8cca05edf87cc74f9e3be7905f538a59762
08/06/2022 13:05:39 — DEBUG — urllib3.connectionpool — Starting new HTTPS connection (1): huggingface.co:443
08/06/2022 13:05:40 — DEBUG — urllib3.connectionpool — https://huggingface.co:443 "HEAD /EleutherAI/gpt-neo-125M/resolve/main/config.json HTTP/1.1" 200 0
08/06/2022 13:05:40 — INFO — aitextgen — GPTNeo loaded with 125M parameters.


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/w3wvHhR.png)

Upload **any smaller text file** (for example, [a text file of Shakespeare plays](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt)) and update the file name in the cell below, then run the cell.

In [ ]:
data_root_dir = gdrive_root_dir + '/aitextgen/training_data'
datasets = ['articles']
dataset_splits = [9]
dataset_iterations = [100]
dataset_runs = [10]
dataset_learnrates = [1e-3]
save_partial = True

file_basename = 'dataset_cache'
file_ext = '.tar.gz'
from_cache = True

If your text file is large (>10MB), it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

Additionally, you may want to consider [compressing the dataset to a cache first](https://docs.aitextgen.io/dataset/) on your local computer, then uploading the resulting `dataset_cache.tar.gz` and setting the `file_name`in the previous cell to that.

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2 in aitextgen. It runs for `num_steps`, and a progress bar will appear to show training progress, current loss (the lower the better the model), and average loss (to give a sense on loss trajectory).

The model will be saved every `save_every` steps in `trained_model` by default, and when training completes. If you mounted your Google Drive, the model will _also_ be saved there in a unique folder.

The training might time out after 4ish hours; if you did not mount to Google Drive, make sure you end training and save the results so you don't lose them! (if this happens frequently, you may want to consider using [Colab Pro](https://colab.research.google.com/signup))

Important parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells
- **`fp16`**: Enables half-precision training for faster/more memory-efficient training. Only works on a T4 or V100 GPU.

Here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: Learning rate of the model training.
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM. (if using `fp16`, you can increase the batch size more safely)

In [ ]:
model_root_dir = gdrive_root_dir + '/aitextgen/models/' + run_id

if save_partial:
    model_partial_dir = model_root_dir + '/partial'
    partial_models = []

In [ ]:
for i in range(len(datasets)):
    file_basepath = data_root_dir + '/' + datasets[i] + '/' + file_basename

    for j in range(dataset_runs[i]):
        if save_partial and (i < len(datasets) - 1 or j < dataset_runs[i] - 1):
            partial_model_name = datasets[i] + '_trainrun-' + str(j + 1)
            model_dir = model_partial_dir + '/' + partial_model_name
            partial_models.append(partial_model_name)
        else:
            model_dir = model_root_dir

        for k in range(dataset_splits[i]):
            if dataset_splits[i] > 1:
                current_file = file_basepath + '.' + str(k) + file_ext
            else:
                current_file = file_basepath + file_ext
            ai.train(current_file,
                     line_by_line=False,
                     from_cache=from_cache,
                     num_steps=dataset_iterations[i],
                     generate_every=dataset_iterations[i],
                     save_every=dataset_iterations[i],
                     save_gdrive=False,
                     run_id=run_id,
                     output_dir=model_dir,
                     learning_rate=dataset_learnrates[i],
                     fp16=False,
                     batch_size=1)

            # R.B.: required to prevent memory leaks in Colab
            gc.collect()

08/06/2022 13:05:56 — INFO — aitextgen — Loading text from /content/drive/My Drive/aitextgen/training_data/articles/dataset_cache.0.tar.gz with generation length of 2048.
08/06/2022 13:05:58 — INFO — aitextgen.TokenDataset — TokenDataset containing 2,296,381 subsets loaded via cache.
08/06/2022 13:05:58 — INFO — torch.distributed.nn.jit.instantiator — Created a temporary directory at /tmp/tmp_oy6_2dy
08/06/2022 13:05:58 — INFO — torch.distributed.nn.jit.instantiator — Writing /tmp/tmp_oy6_2dy/_remote_module_non_sriptable.py
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callbac

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2282: LightningDeprecationWarning: `trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7. Use `ProgressBarBase.get_metrics` instead.
  "`trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7."


100 steps reached: saving model to //content/drive/My Drive/aitextgen/models/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18/partial/articles_trainrun-1
100 steps reached: generating sample texts.
 the first of the “b” mentioned in that post.

“The b” thalker is one of several people who claimed to be involved in this case. And for those that are not so far, I believe the b thalker is not very one of the four people to be involved in this case.”


The b thalker — I’m not the only one who was curious about his past. I’ve seen a post by his father — “I was a young man,” he wrote in “My father called me to find out the source of his father’s father’s father’s father’s father’s father’s father’s father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’ father’


08/06/2022 13:07:21 — INFO — aitextgen — Saving trained model pytorch_model.bin to //content/drive/My Drive/aitextgen/models/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18/partial/articles_trainrun-1
08/06/2022 13:07:23 — INFO — aitextgen — Loading text from /content/drive/My Drive/aitextgen/training_data/articles/dataset_cache.1.tar.gz with generation length of 2048.
08/06/2022 13:07:25 — INFO — aitextgen.TokenDataset — TokenDataset containing 2,290,276 subsets loaded via cache.
08/06/2022 13:07:25 — WARNING — aitextgen — pytorch_model.bin already exists in //content/drive/My Drive/aitextgen/models/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18/partial/articles_trainrun-1 and will be overwritten!
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

100 steps reached: saving model to //content/drive/My Drive/aitextgen/models/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18
100 steps reached: generating sample texts.
. On a whole scale, you can take a bit of your own patch to the bottom of the left, and you can just turn it off again.

→  With the official patch release date, you have to get the patch from the official site.  The only patch in the patch is for all Windows XP users, too.  The kernel update is available to all Windows XP users.  The new update is available for Windows XP users.  The fix is available for Windows XP users.

The updates are available to versions 7.5 and 7.6 of Windows XP.  The new update fixes one critical vulnerability.

There are some interesting data that may be useful for Windows users.  It’s worth noting that Windows XP users are not affected by Windows XP 7.

The worst of all, of course, is that Windows XP users are not affected by Windows XP.

The worst of all, of course, is that 

08/06/2022 15:21:30 — INFO — aitextgen — Saving trained model pytorch_model.bin to //content/drive/My Drive/aitextgen/models/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18
08/06/2022 15:21:33 — INFO — aitextgen — Loading text from /content/drive/My Drive/aitextgen/training_data/articles/dataset_cache.8.tar.gz with generation length of 2048.
08/06/2022 15:21:33 — INFO — aitextgen.TokenDataset — TokenDataset containing 1,898,245 subsets loaded via cache.
08/06/2022 15:21:33 — WARNING — aitextgen — pytorch_model.bin already exists in //content/drive/My Drive/aitextgen/models/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18 and will be overwritten!
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


  0%|          | 0/100 [00:00<?, ?it/s]

100 steps reached: saving model to //content/drive/My Drive/aitextgen/models/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18
100 steps reached: generating sample texts.
-looking-to-be-wearing, and in a little more complex it is a high-risk attack, which is clearly a very low-risk threat, but it seems to be possible to be more likely to be surprised by the fact that you will be surprised by this latest attack.

In the past, we have seen attacks against web sites using the Sality and Sality (Troj/Sality and Sality). These attacks are clearly nothing more sophisticated than a new wave of web pages all the time, but they were rather very low-tech.

It’s not clear what web sites are being hacked, but it’s clear that it is the very latest of the two-thirds of the Sality.

So how do you know that the attacks against web pages are not that sophisticated? Most Web pages are not that sophisticated. They are very few malicious, but they’re a very recent, many more complex, and th

08/06/2022 15:23:01 — INFO — aitextgen — Saving trained model pytorch_model.bin to //content/drive/My Drive/aitextgen/models/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18


You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text.

**If you just trained a model**, you'll get much faster training performance if you reload the model; the next cell will reload the model you just trained from the `trained_model` folder.

`generate()` without any parameters generates a single text from the loaded model to the console.

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = ai.generate_one()`

You can also pass in a `prompt` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `n`. You can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 50 for `batch_size` to avoid going OOM).

Other optional-but-helpful parameters for `ai.generate()` and friends:

*  **`min length`**: The minimum length of the generated text: if the text is shorter than this value after cleanup, aitextgen will generate another one.
*  **`max_length`**: Number of tokens to generate (default 256, you can generate up to 1024 tokens with GPT-2 and 2048 with GPT Neo)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

For bulk generation, you can generate a large amount of texts to a file and sort out the samples locally on your computer. The next cell will generate `num_files` files, each with `n` texts and whatever other parameters you would pass to `generate()`. The files can then be downloaded from the Files sidebar!

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
num_outputs = 2
max_length = 500
temperature = 1.0
top_p = 0.9

prompts = ['Digital Forensics Analysis Report\n',
           'This report is ',
           'The contents of ',
           'Conclusion\n']

In [ ]:
generate_partial_outputs = True

num_outputs_partial = 2
max_length_partial = 500
prompts_partial = ['Digital Forensics Analysis Report\n',
                   'This report is ',
                   'The contents of ',
                   'Conclusion\n']

In [ ]:
output_root_dir = gdrive_root_dir + '/aitextgen/outputs/' + run_id
output_ext = '.txt'

if not os.path.exists(output_root_dir):
    os.makedirs(output_root_dir)

In [ ]:
if save_partial and generate_partial_outputs:
    output_partial_dir = output_root_dir + '/partial'

    for i in partial_models:
        model_dir = model_partial_dir + '/' + i
        ai = aitextgen.aitextgen(model_folder=model_dir, to_gpu=True)

        output_dir = output_partial_dir + '/' + i
        partial_id = run_id + '_partial_' + i
        output_basepath = output_dir + '/' + partial_id + '_output'

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        for j in range(len(prompts_partial)):
            if len(prompts_partial) > 1:
                current_output = output_basepath + '.' + str(j) + output_ext
            else:
                current_output = output_basepath + output_ext
            ai.generate_to_file(n=num_outputs_partial,
                                batch_size=1,
                                prompt=prompts_partial[j],
                                max_length=max_length_partial,
                                temperature=temperature,
                                top_p=top_p,
                                destination_path=current_output)

08/06/2022 15:23:04 — INFO — aitextgen — Loading model from provided weights and config in //content/drive/My Drive/aitextgen/models/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18/partial/articles_trainrun-1.
08/06/2022 15:23:10 — INFO — aitextgen — GPTNeo loaded with 125M parameters.
08/06/2022 15:23:10 — INFO — aitextgen — Using the default GPT-2 Tokenizer.
08/06/2022 15:23:10 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18/partial/articles_trainrun-1/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18_partial_articles_trainrun-1_output.0.txt


  0%|          | 0/2 [00:00<?, ?it/s]

08/06/2022 15:23:24 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18/partial/articles_trainrun-1/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18_partial_articles_trainrun-1_output.1.txt


  0%|          | 0/2 [00:00<?, ?it/s]

08/06/2022 15:23:35 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18/partial/articles_trainrun-1/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18_partial_articles_trainrun-1_output.2.txt


...

  0%|          | 0/2 [00:00<?, ?it/s]

08/06/2022 15:30:21 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18/partial/articles_trainrun-9/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18_partial_articles_trainrun-9_output.2.txt


  0%|          | 0/2 [00:00<?, ?it/s]

08/06/2022 15:30:32 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18/partial/articles_trainrun-9/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18_partial_articles_trainrun-9_output.3.txt


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
if len(datasets) > 0:
    ai = aitextgen.aitextgen(model_folder=model_root_dir, to_gpu=True)

output_basepath = output_root_dir + '/' + run_id + '_output'

for i in range(len(prompts)):
    if len(prompts) > 1:
        current_output = output_basepath + '.' + str(i) + output_ext
    else:
        current_output = output_basepath + output_ext
    ai.generate_to_file(n=num_outputs,
                        batch_size=1,
                        prompt=prompts[i],
                        max_length=max_length,
                        temperature=temperature,
                        top_p=top_p,
                        destination_path=current_output)

08/06/2022 15:30:44 — INFO — aitextgen — Loading model from provided weights and config in //content/drive/My Drive/aitextgen/models/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18.
08/06/2022 15:30:49 — INFO — aitextgen — GPTNeo loaded with 125M parameters.
08/06/2022 15:30:49 — INFO — aitextgen — Using the default GPT-2 Tokenizer.
08/06/2022 15:30:49 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18_output.0.txt


  0%|          | 0/2 [00:00<?, ?it/s]

08/06/2022 15:31:00 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18_output.1.txt


  0%|          | 0/2 [00:00<?, ?it/s]

08/06/2022 15:31:11 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18_output.2.txt


  0%|          | 0/2 [00:00<?, ?it/s]

08/06/2022 15:31:22 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18/aitextgen-CCS-gpt-neo-125M-articles-9000_run_2022-06-08-13-04-18_output.3.txt


  0%|          | 0/2 [00:00<?, ?it/s]

# LICENSE

MIT License

Copyright (c) 2020-2021 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.